In [11]:
%loadFromPOM pom.xml
%classpath C:\\Project\\HAPI-fhir-Lab

In [10]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import ca.uhn.fhir.validation.*;
import org.hl7.fhir.r4.model.*;
import org.hl7.fhir.common.hapi.validation.validator.*;
import org.hl7.fhir.common.hapi.validation.support.*;
import ca.uhn.fhir.context.support.*;
import java.nio.file.*;



In [ ]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

In [ ]:
// Create a validator and register it with the context
FhirValidator validator = ctx.newValidator();

// Create a validation module and register itwith the validator
//IValidatorModule module = new FhirInstanceValidator(ctx);

// Register the module with the validator
//validator.registerValidatorModule(module);

NpmPackageValidationSupport npmPackageSupport = new NpmPackageValidationSupport(ctx);
npmPackageSupport.loadPackageFromClasspath("classpath:profiles/tw-core-ig-v0.1.1.tgz");

// Create a validation support chain
ValidationSupportChain validationSupportChain = new ValidationSupportChain(
    npmPackageSupport,
    new DefaultProfileValidationSupport(ctx),
    new InMemoryTerminologyServerValidationSupport(ctx),
    new CommonCodeSystemsTerminologyService(ctx),
    new SnapshotGeneratingValidationSupport(ctx));

// Set the chain on the validator
FhirInstanceValidator instanceValidator = new FhirInstanceValidator(validationSupportChain);
validator.registerValidatorModule(instanceValidator);

In [21]:
public void readAndValidateFhirResource(FhirContext ctx, FhirValidator validator, String filePath, Map<String, String> resourceProfiles, Map<String, Boolean> resourceValidated) {
    try {
        // Read FHIR data
        String json = new String(Files.readAllBytes(Paths.get(filePath)));

        // Parse JSON-encoded FHIR resource
        Bundle bundle = ctx.newJsonParser().parseResource(Bundle.class, json);

        // From Bundle, read data
        ValidationResult result = null;
        String resourceType = null;
        List resourceTypes = new ArrayList();
        for (Bundle.BundleEntryComponent entry : bundle.getEntry()) {
            result = null;
            resourceType = entry.getResource().getResourceType().name();

            if (resourceProfiles.containsKey(resourceType)) {
                DomainResource resource = (DomainResource) entry.getResource();
                if (resourceValidated.get(resourceType) == false) {
                    resourceValidated.put(resourceType, true);
                    resource.getMeta().addProfile(resourceProfiles.get(resourceType));
                    result = validator.validateWithResult(resource);
                }
            } else {
                if (!resourceTypes.contains(resourceType)) {
                    resourceTypes.add(resourceType);
                    System.out.println("\nTW Core IG尚未定義-" + resourceType);
                }
            }

            // Print the validation result
            if (result != null) {
                if (result.isSuccessful()) {
                    System.out.println("\nValidation passed for " + resourceType);
                } else {
                    System.out.println("\nValidation failed for" + " " + resourceType);
                    result.getMessages().forEach(msg -> System.out.println(msg.getLocationString() + " " + msg.getMessage()));
                }
            }
        }
    } catch (Exception e) {
        e.printStackTrace();
    }
}

In [22]:
Map<String, String> resourceProfiles = new HashMap<>();
resourceProfiles.put("Patient", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore");
resourceProfiles.put("Observation", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Observation-vitalSigns-twcore");
resourceProfiles.put("MedicationRequest", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/MedicationRequest-twcore");
resourceProfiles.put("Medication", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Medication-twcore");
resourceProfiles.put("Encounter", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Encounter-twcore");
resourceProfiles.put("Condition", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Condition-twcore");
resourceProfiles.put("DiagnosticReport", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/DiagnosticReport-twcore");
resourceProfiles.put("Procedure", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Procedure-twcore");

// 資源類型和對應的 profile
Map<String,Boolean> resourceValidated = new HashMap<>();
resourceValidated.put("Patient", false);
resourceValidated.put("Observation", false);
resourceValidated.put("MedicationRequest", false);
resourceValidated.put("Medication", false);
resourceValidated.put("Encounter", false);
resourceValidated.put("Condition", false);
resourceValidated.put("DiagnosticReport", false);
resourceValidated.put("Procedure", false);

// call readAndValidateFhirResource
readAndValidateFhirResource(ctx, validator, "C:\\Project\\HAPI-fhir-Lab\\synthetic-data\\fhir\\Alejandro916_Espinoza81.json", resourceProfiles, resourceValidated);

resourceProfiles.forEach((key, value) -> {
    if (resourceValidated.get(key) == false) {
        System.out.println("\n\n合成資料沒有產生：" + key);
    }
});


Validation failed for Patient
Patient.extension[0] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.extension[1] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.extension[2] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.extension[3] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.identifier[0] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient.identifier[2] This element does not match any known slice defined in the profile https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore
Patient

In [24]:
Map<String, String> resourceProfiles = new HashMap<>();
resourceProfiles.put("Organization", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Organization-hosp-twcore");

// 資源類型和對應的 profile
Map<String,Boolean> resourceValidated = new HashMap<>();
resourceValidated.put("Organization", false);

// call readAndValidateFhirResource
readAndValidateFhirResource(ctx, validator, "C:\\Project\\HAPI-fhir-Lab\\synthetic-data\\fhir\\hospitalInformation.json", resourceProfiles, resourceValidated);


Validation failed for Organization
Organization.extension[0] Unknown extension http://synthetichealth.github.io/synthea/utilization-encounters-extension
Organization.extension[1] Unknown extension http://synthetichealth.github.io/synthea/utilization-procedures-extension
Organization.extension[2] Unknown extension http://synthetichealth.github.io/synthea/utilization-labs-extension
Organization.extension[3] Unknown extension http://synthetichealth.github.io/synthea/utilization-prescriptions-extension
Organization.identifier[0].system Value is 'https://github.com/synthetichealth/synthea' but must be 'https://twcore.mohw.gov.tw/ig/twcore/CodeSystem/organization-identifier-tw'

TW Core IG尚未定義-Location


In [25]:
Map<String, String> resourceProfiles = new HashMap<>();
resourceProfiles.put("Practitioner", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Practitioner-twcore");

// 資源類型和對應的 profile
Map<String,Boolean> resourceValidated = new HashMap<>();
resourceValidated.put("Practitioner", false);

// call readAndValidateFhirResource
readAndValidateFhirResource(ctx, validator, "C:\\Project\\HAPI-fhir-Lab\\synthetic-data\\fhir\\practitionerInformation.json", resourceProfiles, resourceValidated);



Validation passed for Practitioner
